<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/PSO_vs_ED_vs_MMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

## Definición de función Alpine.

In [ ]:
def alpine(x, y):
    """Calcula el valor de la funcion Alpine."""
    return np.abs(x * np.sin(x) + 0.1 * x) + np.abs(y * np.sin(y) + 0.1 * y)

## Implementación de PSO

In [ ]:
def pso(max_iter, n_particulas, w, c1, c2, rango_x=(-5, 5), rango_y=(-5, 5)):
    """Optimiza la funcion Alpine usando PSO."""

    # Inicializar posiciones aleatorias en el rango especificado
    particulas = [
        (random.uniform(rango_x[0], rango_x[1]),
         random.uniform(rango_y[0], rango_y[1]))
        for _ in range(n_particulas)
    ]

    # Inicializar velocidades pequenas
    velocidades = [
        (random.uniform(-1, 1), random.uniform(-1, 1))
        for _ in range(n_particulas)
    ]
    # Evaluar funcion objetivo para cada particula
    valores = [alpine(x, y) for x, y in particulas]

    # Mejor posicion personal (pbest): inicialmente es la posicion actual
    pbest = particulas[:]  # Copia de las posiciones
    pbest_valores = valores[:]  # Copia de los valores

    # Mejor posicion global (gbest): la mejor entre todas las particulas
    indice_mejor = np.argmin(valores)
    gbest = particulas[indice_mejor]
    gbest_valor = valores[indice_mejor]

    # Guardar historial para graficar convergencia
    historial_valores = [gbest_valor]

    # Guardar coordenadas de todas las particulas en cada iteracion
    coordenadas_x = [np.array([p[0] for p in particulas])]
    coordenadas_y = [np.array([p[1] for p in particulas])]

    for iteracion in range(max_iter):
        for i in range(n_particulas):
            # Posicion y velocidad actual de la particula i
            x, y = particulas[i]
            vx, vy = velocidades[i]

            # Generar numeros aleatorios para estocasticidad
            r1 = random.random()
            r2 = random.random()

            # Actualizar velocidad segun ecuacion de PSO
            vx_nueva = (w * vx +
                        c1 * r1 * (pbest[i][0] - x) +
                        c2 * r2 * (gbest[0] - x))

            vy_nueva = (w * vy +
                        c1 * r1 * (pbest[i][1] - y) +
                        c2 * r2 * (gbest[1] - y))

            # Actualizar posicion: nueva_posicion = posicion + velocidad
            x_nueva = x + vx_nueva
            y_nueva = y + vy_nueva

            # Asegurar que la posicion este dentro del rango permitido
            x_nueva = np.clip(x_nueva, rango_x[0], rango_x[1])
            y_nueva = np.clip(y_nueva, rango_y[0], rango_y[1])

            # Guardar nueva posicion y velocidad
            particulas[i] = (x_nueva, y_nueva)
            velocidades[i] = (vx_nueva, vy_nueva)

            # Evaluar funcion objetivo en la nueva posicion
            valor_nuevo = alpine(x_nueva, y_nueva)
            valores[i] = valor_nuevo

            # Actualizar pbest si mejoramos la mejor posicion personal
            if valor_nuevo < pbest_valores[i]:
                pbest[i] = (x_nueva, y_nueva)
                pbest_valores[i] = valor_nuevo

            # Actualizar gbest si mejoramos la mejor posicion global
            if valor_nuevo < gbest_valor:
                gbest = (x_nueva, y_nueva)
                gbest_valor = valor_nuevo

            # Guardar el mejor valor de esta iteracion (para grafica de convergencia)
        historial_valores.append(gbest_valor)

        # Guardar coordenadas de todas las particulas (para animacion)
        coordenadas_x.append(np.array([p[0] for p in particulas]))
        coordenadas_y.append(np.array([p[1] for p in particulas]))

    # Retornar: mejor solucion, historial y coordenadas
    return (gbest[0], gbest[1], gbest_valor,
            historial_valores, coordenadas_x, coordenadas_y)

## Implementación de ED

In [ ]:
def evolucion_diferencial(max_iter, tam_poblacion, F, CR,
                          rango_x=(-5, 5), rango_y=(-5, 5)):
    """Optimiza la funcion Alpine usando evolucion diferencial."""
    # Inicializar poblacion con individuos aleatorios
    poblacion = [
        (random.uniform(rango_x[0], rango_x[1]),
         random.uniform(rango_y[0], rango_y[1]))
        for _ in range(tam_poblacion)
    ]

    # Evaluar funcion objetivo para cada individuo
    costos = [alpine(x, y) for x, y in poblacion]

    # Identificar mejor individuo de la poblacion inicial
    indice_mejor = np.argmin(costos)
    mejor_individuo = poblacion[indice_mejor]
    mejor_valor = costos[indice_mejor]

    # Preparar historial para analisis
    historial_valores = [mejor_valor]
    coordenadas_x = [np.array([ind[0] for ind in poblacion])]
    coordenadas_y = [np.array([ind[1] for ind in poblacion])]

    for generacion in range(max_iter):
        for i in range(tam_poblacion):
            # Seleccionar tres individuos aleatorios diferentes
            # r1, r2, r3 deben ser distintos entre si y distintos de i
            indices_disponibles = [j for j in range(tam_poblacion) if j != i]
            r1, r2, r3 = random.sample(indices_disponibles, 3)

            # MUTACION: Generar vector donante (mutante)
            vector_base = np.array(poblacion[r1])
            diferencia = np.array(poblacion[r2]) - np.array(poblacion[r3])
            vector_donante = vector_base + F * diferencia

            # CRUZA BINOMIAL: Combinar vector objetivo con vector donante
            # Seleccionar al menos una coordenada del donante
            coor_obligatoria = random.randint(0, 1)  # 0 para x, 1 para y

            # Crear vector de prueba (hijo)
            vector_prueba = []

            for j in range(2):  # Iterar sobre las dos dimensiones (x, y)
                # Heredar del donante si: cumple CR O es la coordenada obligatoria
                if random.random() < CR or j == coor_obligatoria:
                    vector_prueba.append(vector_donante[j])
                else:
                    # Heredar del objetivo
                    vector_prueba.append(poblacion[i][j])

            # Extraer coordenadas del vector de prueba
            x_prueba, y_prueba = vector_prueba[0], vector_prueba[1]

            # Asegurar que el vector de prueba este dentro de los limites
            x_prueba = np.clip(x_prueba, rango_x[0], rango_x[1])
            y_prueba = np.clip(y_prueba, rango_y[0], rango_y[1])

            # Evaluar funcion objetivo del vector de prueba
            valor_prueba = alpine(x_prueba, y_prueba)

            # SELECCION: Reemplazar si el hijo es igual o mejor que el padre
            if valor_prueba <= costos[i]:
                poblacion[i] = (x_prueba, y_prueba)
                costos[i] = valor_prueba

        # Actualizar mejor solucion de la generacion
        indice_mejor = np.argmin(costos)
        mejor_individuo = poblacion[indice_mejor]
        mejor_valor = costos[indice_mejor]

        # Guardar historial
        historial_valores.append(mejor_valor)
        coordenadas_x.append(np.array([ind[0] for ind in poblacion]))
        coordenadas_y.append(np.array([ind[1] for ind in poblacion]))

        # Retornar mejor solucion encontrada y su historial
    return (mejor_individuo[0], mejor_individuo[1], mejor_valor,
            historial_valores, coordenadas_x, coordenadas_y)

## Implementación de MMC.

In [ ]:
def crear_compositor(melodias_por_comp, rango_x, rango_y):
    """
    Crea un compositor para MMC generando melodias iniciales
    dentro del espacio de busqueda.
    """

    melodias = []
    valores = []

    # Generar melodias iniciales aleatorias
    for _ in range(melodias_por_comp):
        x = random.uniform(rango_x[0], rango_x[1])
        y = random.uniform(rango_y[0], rango_y[1])
        melodia = (x, y)

        # Evaluar la melodia
        valor = alpine(x, y)

        melodias.append(melodia)
        valores.append(valor)

    # Identificar la mejor melodia inicial
    indice_mejor = np.argmin(valores)
    mejor_melodia = melodias[indice_mejor]
    mejor_valor = valores[indice_mejor]

    # Crear estructura del compositor
    compositor = {
        "melodias": melodias,
        "valores": valores,
        "mejor_melodia": mejor_melodia,
        "mejor_valor": mejor_valor
    }

    return compositor

def generar_melodia(compositor, mejor_vecino,
                    rango_x, rango_y, alpha=0.5, beta=0.5):
    """
    Genera una nueva melodia combinando:
    - mejor melodia propia,
    - mejor melodia del vecino,
    - perturbacion aleatoria.
    """

    # Extraer mejor melodia propia
    x_propio, y_propio = compositor["mejor_melodia"]

    # Extraer mejor melodia del vecino
    x_vecino, y_vecino = mejor_vecino

    # Combinacion lineal segun la ecuacion de MMC
    x_combinado = alpha * x_propio + beta * x_vecino
    y_combinado = alpha * y_propio + beta * y_vecino


    # Magnitud de perturbacion: 1% del rango de cada eje
    magnitud_x = 0.01 * (rango_x[1] - rango_x[0])
    magnitud_y = 0.01 * (rango_y[1] - rango_y[0])

    # Agregar perturbacion uniforme
    x_nueva = x_combinado + random.uniform(-magnitud_x, magnitud_x)
    y_nueva = y_combinado + random.uniform(-magnitud_y, magnitud_y)

    # Ajustar al borde mas cercano si se sale del rango
    x_nueva = np.clip(x_nueva, rango_x[0], rango_x[1])
    y_nueva = np.clip(y_nueva, rango_y[0], rango_y[1])

    return (x_nueva, y_nueva)

def actualizar_partituras(compositor, melodia_nueva, valor_nuevo):
    """
    Actualiza la matriz de partituras del compositor.
    Reemplaza a la peor melodia si la nueva es mejor.
    """
    valores = compositor["valores"]
    melodias = compositor["melodias"]

    # Encontrar el peor valor (maximo en minimizacion)
    indice_peor = np.argmax(valores)
    valor_peor = valores[indice_peor]

    # Reemplazo elitista: solo aceptar si mejora
    if valor_nuevo < valor_peor:
        melodias[indice_peor] = melodia_nueva
        valores[indice_peor] = valor_nuevo

        # Actualizar mejor melodia personal
        if valor_nuevo < compositor["mejor_valor"]:
            compositor["mejor_valor"] = valor_nuevo
            compositor["mejor_melodia"] = melodia_nueva

def mmc(max_iter, num_compositores, melodias_por_comp,
        alpha=0.5, beta=0.5, rango_x=(-5, 5), rango_y=(-5, 5)):
    """Ejecuta el Metodo de Composicion Musical (MMC) sobre la funcion Alpine."""

    # Crear la sociedad inicial de compositores
    compositores = [
        crear_compositor(melodias_por_comp, rango_x, rango_y)
        for _ in range(num_compositores)
    ]

    # Mejor solucion global inicial (gbest)
    mejor_global = min(
        compositores,
        key=lambda c: c["mejor_valor"]
    )
    gbest = mejor_global["mejor_melodia"]
    gbest_valor = mejor_global["mejor_valor"]

    # Historial de valores para analizar la convergencia
    historial_valores = [gbest_valor]

    # Guardar coordenadas para visualizacion (opcional)
    coordenadas_x = [
        np.array([c["mejor_melodia"][0] for c in compositores])
    ]
    coordenadas_y = [
        np.array([c["mejor_melodia"][1] for c in compositores])
    ]

    # Ciclo principal de MMC
    for iteracion in range(max_iter):

        # Cada compositor genera y evalúa una nueva melodía
        for i, compositor in enumerate(compositores):

            # Topología completa: elegir un vecino aleatorio distinto
            indices_vecinos = [j for j in range(num_compositores) if j != i]
            indice_vecino = random.choice(indices_vecinos)
            vecino = compositores[indice_vecino]

            # Obtener la mejor melodía del vecino
            mejor_vecino = vecino["mejor_melodia"]

            # Generar nueva melodía candidata
            candidata = generar_melodia(
                compositor,
                mejor_vecino=mejor_vecino,
                alpha=alpha,
                beta=beta,
                rango_x=rango_x,
                rango_y=rango_y
            )

            # Evaluar la candidata con la función objetivo
            x_cand, y_cand = candidata
            valor_cand = alpine(x_cand, y_cand)
            # Actualizar matriz de partituras (elitismo)
            actualizar_partituras(compositor, candidata, valor_cand)

        # Actualizar la mejor solución global si corresponde
        if valor_cand < gbest_valor:
            gbest = candidata
            gbest_valor = valor_cand

        # Guardar historial para convergencia y visualización (opcional)
        historial_valores.append(gbest_valor)
        coordenadas_x.append(np.array([c["mejor_melodia"][0] for c in compositores]))
        coordenadas_y.append(np.array([c["mejor_melodia"][1] for c in compositores]))

    # Devolver resultados al completar todas las iteraciones
    x_opt, y_opt = gbest
    return (x_opt, y_opt, gbest_valor,
            historial_valores, coordenadas_x, coordenadas_y)


## Configuración de parámeteros para el experimento comparativo: PSO, ED y MMC

In [ ]:
# Configuracion del experimento
num_ejecuciones = 100
np.random.seed(42)  # Para reproducibilidad

# Configuracion de PSO (1010 evaluaciones)
config_pso = {
    'max_iter': 100,
    'n_particulas': 10,
    'w': 0.75,
    'c1': 0.50,
    'c2': 0.50
}

# Configuracion de Evolucion Diferencial (1020 evaluaciones)
config_ed = {
    'max_iter': 50,
    'tam_poblacion': 20,
    'F': 0.5,
    'CR': 0.7
}

# Configuracion de MMC (1005 evaluaciones)
config_mmc = {
    'max_iter': 100,
    'num_compositores': 10,
    'melodias_por_comp': 5,
    'alpha': 0.1,
    'beta': 0.1
}

# Almacenar resultados
costos_pso = []
costos_ed = []
costos_mmc = []
distancias_pso = []
distancias_ed = []
distancias_mmc = []

print("Ejecutando experimento comparativo...")
print("="*60)

for i in range(num_ejecuciones):
    # Ejecutar PSO
    x_pso, y_pso, costo_pso, _, _, _ = pso(
        max_iter=config_pso['max_iter'],
        n_particulas=config_pso['n_particulas'],
        w=config_pso['w'],
        c1=config_pso['c1'],
        c2=config_pso['c2']
    )

    # Ejecutar Evolucion Diferencial
    x_ed, y_ed, costo_ed, _, _, _ = evolucion_diferencial(
        max_iter=config_ed['max_iter'],
        tam_poblacion=config_ed['tam_poblacion'],
        F=config_ed['F'],
        CR=config_ed['CR']
    )

    # Ejecutar MMC
    x_mmc, y_mmc, costo_mmc, _, _, _ = mmc(
        max_iter=config_mmc['max_iter'],
        num_compositores=config_mmc['num_compositores'],
        melodias_por_comp=config_mmc['melodias_por_comp'],
        alpha=config_mmc['alpha'],
        beta=config_mmc['beta']
    )

    # Guardar resultados
    costos_pso.append(costo_pso)
    costos_ed.append(costo_ed)
    costos_mmc.append(costo_mmc)

    distancias_pso.append(np.sqrt(x_pso**2 + y_pso**2))
    distancias_ed.append(np.sqrt(x_ed**2 + y_ed**2))
    distancias_mmc.append(np.sqrt(x_mmc**2 + y_mmc**2))

    # Mostrar progreso
    if (i + 1) % 10 == 0:
        print(f"Completadas {i + 1}/{num_ejecuciones} ejecuciones")

print("\nExperimento completado!")

## Análisis estadístico de resultados.

In [ ]:
# Calcular estadisticos para cada algoritmo
print("="*70)
print("ANALISIS ESTADISTICO COMPARATIVO")
print("="*70)

print("\n" + "="*70)
print("PSO (OPTIMIZACION POR ENJAMBRE DE PARTICULAS)")
print("="*70)
print(f"Costos encontrados:")
print(f"  Media:              {np.mean(costos_pso):.6f}")
print(f"  Mediana:            {np.median(costos_pso):.6f}")
print(f"  Desviacion estandar: {np.std(costos_pso, ddof=1):.6f}")
print(f"  Minimo:             {np.min(costos_pso):.6f}")
print(f"  Maximo:             {np.max(costos_pso):.6f}")

cuartiles_pso = np.percentile(costos_pso, [25, 50, 75])
print(f"  Cuartiles (Q1,Q2,Q3): [{cuartiles_pso[0]:.4f}, "
      f"{cuartiles_pso[1]:.4f}, {cuartiles_pso[2]:.4f}]")

print("\n" + "="*70)
print("EVOLUCION DIFERENCIAL")
print("="*70)
print(f"Costos encontrados:")
print(f"  Media:              {np.mean(costos_ed):.6f}")
print(f"  Mediana:            {np.median(costos_ed):.6f}")
print(f"  Desviacion estandar: {np.std(costos_ed, ddof=1):.6f}")
print(f"  Minimo:             {np.min(costos_ed):.6f}")
print(f"  Maximo:             {np.max(costos_ed):.6f}")

cuartiles_ed = np.percentile(costos_ed, [25, 50, 75])
print(f"  Cuartiles (Q1,Q2,Q3): [{cuartiles_ed[0]:.4f}, "
      f"{cuartiles_ed[1]:.4f}, {cuartiles_ed[2]:.4f}]")

print("\n" + "="*70)
print("METODO DE COMPOSICION MUSICAL (MMC)")
print("="*70)
print(f"Costos encontrados:")
print(f"  Media:              {np.mean(costos_mmc):.6f}")
print(f"  Mediana:            {np.median(costos_mmc):.6f}")
print(f"  Desviacion estandar: {np.std(costos_mmc, ddof=1):.6f}")
print(f"  Minimo:             {np.min(costos_mmc):.6f}")
print(f"  Maximo:             {np.max(costos_mmc):.6f}")

cuartiles_mmc = np.percentile(costos_mmc, [25, 50, 75])
print(f"  Cuartiles (Q1,Q2,Q3): [{cuartiles_mmc[0]:.4f}, "
      f"{cuartiles_mmc[1]:.4f}, {cuartiles_mmc[2]:.4f}]")


## Diagramas de cajas

In [ ]:
# Crear diagrama de cajas
datos = [costos_pso, costos_ed, costos_mmc]
etiquetas = ['PSO', 'Evolucion\nDiferencial', 'MMC']

plt.figure(figsize=(12, 7))

# Crear boxplot
bp = plt.boxplot(datos,
                 labels=etiquetas,
                 patch_artist=True,
                 widths=0.6)

# Personalizar colores
colores = ['lightcoral', 'lightgreen', 'lightblue']
for patch, color in zip(bp['boxes'], colores):
    patch.set_facecolor(color)

# Linea del optimo global
plt.axhline(y=0, color='green', linestyle='--', linewidth=2,
            label='Optimo global', alpha=0.7)

# Configuracion
plt.ylabel('Costo de la solucion f(x,y)', fontsize=12)
plt.title('Comparacion de algoritmos poblacionales: PSO, ED y MMC',
          fontsize=14)
plt.grid(True, alpha=0.3, axis='y')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()